In [27]:
#hide
#default_exp tools.change_manager
from nbdev.showdoc import *
from block_types.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Change manager

Displays the relationship between scores obtained across experiments with two different evaluation metrics indicated as argument. Calculates Pearson R-squared coefficient.

In [28]:
#export
import argparse
from importlib import reload
import warnings
import pdb
warnings.filterwarnings('ignore')

In [29]:
#for tests
import pytest
from hpsearch.config.manager_factory import ManagerFactory
from hpsearch.examples.example_experiment_manager import ExampleExperimentManager
from hpsearch.examples.dummy_experiment_manager import DummyExperimentManager

In [30]:
#export
def change_manager (subclass=None):
    import hpsearch.config.manager_factory as mf
    reload (mf)
        
    manager_factory = mf.ManagerFactory (verbose=2)
    
    if subclass is None:
        manager_factory.list_subclasses ()
    else:
        manager_factory.change_manager (subclass)
        manager_factory.import_written_manager ()
        change_manager (None)

### example usage

In [36]:
#export tests.tools.test_change_manager
def test_change_manager_list ():
    # the following lists stored experiment managers, keeping the current experiment manager
    manager_factory = ManagerFactory ()
    manager_factory.delete_and_reset_all ()
    
    print ('\ninitial list')
    em_before = manager_factory.get_experiment_manager ()
    change_manager (None)
    em_after = manager_factory.get_experiment_manager ()
    assert em_before.__class__.__name__ == em_after.__class__.__name__

    # **************************************************************
    # same but using ExampleExperimentManager as registered manager
    # **************************************************************
    em = ExampleExperimentManager()
    manager_factory.register_manager (em)
    manager_factory.write_manager (em)
    
    print ('\nlist after storing ExampleExperimentManager')
    em_before = manager_factory.get_experiment_manager ()
    change_manager (None)
    em_after = manager_factory.get_experiment_manager ()
    assert em_before.__class__.__name__ == em_after.__class__.__name__
    
    # we store a third EM (DummyExperimentManager) and list the two 
    # stored managers
    print ('\nlist after storing DummyExperimentManager')
    em = DummyExperimentManager()
    manager_factory.register_manager(em)
    manager_factory.write_manager(em)
    assert manager_factory.get_experiment_manager().__class__.__name__ == 'DummyExperimentManager'
    change_manager (None)
    
    # we change the registered manager back to the first one
    print ('\nlist after changing manager')
    change_manager('ExampleExperimentManager')
    # TODO: error in pytest
    #assert manager_factory.get_experiment_manager().__class__.__name__ == 'ExampleExperimentManager'

In [37]:
tst.run (test_change_manager_list, tag='dummy')

class2import not found, switching to original method
class2import not found, switching to original method


running test_change_manager_list

initial list
subclasses: dict_keys([])
experiment manager registered: ExperimentManager

list after storing ExampleExperimentManager
subclasses: dict_keys(['ExampleExperimentManager'])
experiment manager registered: ExampleExperimentManager

list after storing DummyExperimentManager
subclasses: dict_keys(['ExampleExperimentManager', 'DummyExperimentManager'])
experiment manager registered: DummyExperimentManager

list after changing manager
subclasses: dict_keys(['ExampleExperimentManager', 'DummyExperimentManager'])
experiment manager registered: ExampleExperimentManager


In [33]:
#export
def main():
    parser = argparse.ArgumentParser(description='change experiment manager')
    parser.add_argument('-m','--manager', type=str, default=None, help="new experiment manager to use")
    parser.add_argument('-l', '--list', action= "store_true", help="list experiment managers registered so far")
    pars = parser.parse_args()
    
    #pdb.set_trace()
    if pars.list:
        change_manager (None)
    else:
        if pars.manager is None:
            print ('you did not indicate any manager to change to, listing the managers available')
        change_manager (pars.manager)